In [19]:
from dotenv import load_dotenv
import os
import chromadb
from pathlib import Path

#.env file load
load_dotenv('../RAG_Pipeline/.env')


HOST = os.getenv('CHROMADB_HOST')
PORT = os.getenv('CHROMADB_PORT')
COLLECTION_NAME = 'A'
UPSTAGE_KEY_TM = os.getenv('UPSTAGE_KEY_TM')

print(HOST)
print(PORT)

client = chromadb.HttpClient(host=HOST, port=PORT)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


mywol.iptime.org
3307


In [ ]:
collection = client.get_or_create_collection('A')

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [ ]:
def check_file_name(file_name): # 파일 이름 docs/로 자른다음 앞에 문자 확인?
    


In [ ]:
collection_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
# 파일 별로 컬렉션 나누는 로직 추가해보기

In [41]:
import re

# 텍스트 전처리 함수
def clean_text(text):
    """텍스트 전처리 개선"""
    # 페이지 번호 제거
    text = re.sub(r'-\s*\d+\s*-', '', text)
    text = re.sub(r'\s*\d+\s*\n', '', text)

    # 불필요한 특수문자 제거
    text = re.sub(r'[•◆▶▷→▪]', '', text)

    # 삭제하고 싶은 특정 문구들
    remove_patterns = [
        r'도로교통법\s*\n',
        r'.*법제처.*\n',
        r'국가법령정보센터\s*\n',
        r'confusion matrix.*?(?=\n|$)',  # confusion matrix 관련 텍스트
        r'[rR][oO][cC].*?(?=\n|$)',     # ROC 관련 텍스트
    ]

    # 모든 패턴을 하나의 정규식으로 결합
    combined_pattern = '|'.join(remove_patterns)
    text = re.sub(combined_pattern, '', text)

    # 불필요한 숫자 패턴 제거
    text = re.sub(r'^\d+\.\s*', '', text, flags=re.MULTILINE)

    # 연속된 공백 정리
    text = re.sub(r'\s+', ' ', text)

    # 불필요한 줄바꿈 정리
    text = re.sub(r'\n+', '\n', text)

    # URL 패턴 정리
    text = re.sub(r'http\S+\s?', '', text)

    return text.strip()

In [ ]:


# 임베딩 모델 생성
embeddings_model = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-MiniLM-L6-v2', # 한국어 자연어 추론에 최적화된 모델
        model_kwargs={'device':'cpu'}, # 모델이 CPU에서 실행되도록 설정 GPU를 사용할 수 있다면 cuda로 설정
        encode_kwargs={'normalize_embeddings':True}, # 임베딩을 정규화하여 모든 벡터가 같은 범위 값을 갖도록 한다.
    )from langchain_community.embeddings import HuggingFaceEmbeddings

In [55]:
from langchain_community.document_loaders.pdf import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyMuPDFLoader("docs/도로교통법.pdf")
pages = loader.load() # 페이지 별 추출 텍스트를 리스트로 반환
    
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,        # 청크 크기
    chunk_overlap=100,      # 청크 간 중복
    length_function=len,
    separators=["\n\n"] # 분할 기준
)

for i, page in enumerate(pages):
    text = clean_text(page.page_content)

    chunk = text_splitter.split_text(text)
    print(chunk[0])

    embedding = embeddings_model.embed_query(chunk[0])
    print(embedding)

    collection.add(
        embeddings = [embedding],
        documents = [chunk[0]],
        ids = [f"chunk_{i}"]
    )

[시행 2018. 4. 25.] [법률 제15530호, 2018. 3. 27., 일부개정] 경찰청 (교통기획계(법제총괄))경찰청 (교통운영계(신호, 안전표지, 도로공사신고))경찰청 (운전면허계(운전면허))경찰청 (교통안전계(안전, 단속, 어린이통학버스))경찰청 (교통조사계(교통사고조사)) 제1장 총칙 제1조(목적) 이 법은 도로에서 일어나는 교통상의 모든 위험과 장해를 방지하고 제거하여 안전하고 원활한 교통을 확 보함을 목적으로 한다. 제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2012. 3. 21., 2013. 3. 23., 2014. 1. 28., 2014. 19., 2017. 3. 21., 2017. 7. 26., 2017. 10. 24.> "도로"란 다음 각 목에 해당하는 곳을 말한다. 가. 「도로법」에 따른 도로 나. 「유료도로법」에 따른 유료도로 다. 「농어촌도로 정비법」에 따른 농어촌도로 라. 그 밖에 현실적으로 불특정 다수의 사람 또는 차마(車馬)가 통행할 수 있도록 공개된 장소로서 안전하고 원활 한 교통을 확보할 필요가 있는 장소 "자동차전용도로"란 자동차만 다닐 수 있도록 설치된 도로를 말한다. "고속도로"란 자동차의 고속 운행에만 사용하기 위하여 지정된 도로를 말한다. "차도"(車道)란 연석선(차도와 보도를 구분하는 돌 등으로 이어진 선을 말한다. 이하 같다), 안전표지 또는 그와 비슷한 인공구조물을 이용하여 경계(境界)를 표시하여 모든 차가 통행할 수 있도록 설치된 도로의 부분을 말한다 . "중앙선"이란 차마의 통행 방향을 명확하게 구분하기 위하여 도로에 황색 실선(實線)이나 황색 점선 등의 안전표 지로 표시한 선 또는 중앙분리대나 울타리 등으로 설치한 시설물을 말한다. 다만, 제14조제1항 후단에 따라 가변 차로(可變車路)가 설치된 경우에는 신호기가 지시하는 진행방향의 가장 왼쪽에 있는 황색 점선을 말한다. "차로"란 차마가 한 줄로 도로의 정하여진 부분을 통행하도록 차선(車線)으로 구분한 차도의 부분을 말한다.

In [56]:
# 사용자 질문 반환 함수
def get_user_query():
    lines = []

    while True:
        line = input("질문을 입력하세요 (종료: 빈 줄에서 Enter)")
        # 빈 줄이고 이전에 입력된 내용이 있으면 종료
        if not line and lines:
            break
        # 입력된 줄이 있으면 추가
        if line:
            lines.append(line)

    return "\n".join(lines)

In [59]:
user_query = get_user_query()

user_query_embedding = embeddings_model.embed_query(user_query)
k = 2

results = collection.query(
    query_embeddings=[user_query_embedding],
    n_results=k,
    include=['documents']
)
print(user_query)

for i, chunk_id in enumerate(results['ids'][0]):
    print(f"{i}번째 청크 ID: {chunk_id}")
    print(f"해당 청크 내용: {results['documents'][0][i]}")  # 내용 앞부분만
    print('-------\n')


무인 교통단속용 장비의 설치 및 관리
0번째 청크 ID: chunk_6
해당 청크 내용: 제4조의2(무인 교통단속용 장비의 설치 및 관리) ① 지방경찰청장, 경찰서장 또는 시장등은 이 법을 위반한 사실을 기 록ㆍ증명하기 위하여 무인(無人) 교통단속용 장비를 설치ㆍ관리할 수 있다. ② 무인 교통단속용 장비의 철거 또는 원상회복 등에 관하여는 제3조제3항부터 제5항까지의 규정을 준용한다. 이 경우 "교통안전시설"은 "무인 교통단속용 장비"로 본다. [전문개정 2011. 6. 8.] 제4조의2(무인 교통단속용 장비의 설치 및 관리) ① 지방경찰청장, 경찰서장 또는 시장등은 이 법을 위반한 사실을 기 록ㆍ증명하기 위하여 무인(無人) 교통단속용 장비를 설치ㆍ관리할 수 있다. ② 무인 교통단속용 장비의 철거 또는 원상회복 등에 관하여는 제3조제4항부터 제6항까지의 규정을 준용한다. 이 경우 "교통안전시설"은 "무인 교통단속용 장비"로 본다.<개정 2018. 6. 12.> [전문개정 2011. 6. 8.] [시행일 : 2019. 6. 13.] 제4조의[시행일 : 2019. 6. 13] 제4조의제5조(신호 또는 지시에 따를 의무) ① 도로를 통행하는 보행자와 차마의 운전자는 교통안전시설이 표시하는 신호 또 는 지시와 다음 각 호의 어느 하나에 해당하는 사람이 하는 신호 또는 지시를 따라야 한다. <개정 2015. 7. 24.> 교통정리를 하는 국가경찰공무원(의무경찰을 포함한다. 이하 같다) 및 제주특별자치도의 자치경찰공무원(이하 "자치경찰공무원"이라 한다) 국가경찰공무원 및 자치경찰공무원(이하 "경찰공무원"이라 한다)을 보조하는 사람으로서 대통령령으로 정하는 사람(이하 "경찰보조자"라 한다) ② 도로를 통행하는 보행자와 모든 차마의 운전자는 제1항에 따른 교통안전시설이 표시하는 신호 또는 지시와 교통 정리를 하는 국가경찰공무원ㆍ자치경찰공무원 또는 경찰보조자(이하 "경찰공무원등"이라 한다)의 신호 또는 지시 가 서로 다른 경우에는 경찰공무원등의 신호 또는 지시에 따라야 한다.

In [61]:
import httpx

contexts = []

for i, doc in enumerate(results['documents'][0]):
    contexts.append(f"{i + 1}번째 관련 문서 : \n{doc}\n")

context_text = "\n".join(contexts)

# 프롬프트 생성
prompt = f"""질문: {user_query}
참고할 도로교통법 내용 :
{context_text}

위 도로교통법 내용만을 참고하여 질문에 답변해주시기 바랍니다.
다른 참고 문서는 참고하지 말아주시기 바랍니다.
"""
api_key = UPSTAGE_KEY_TM
base_url = "https://api.upstage.ai/v1/chat/completions"
payload = {
    "model": "solar-pro",
    "messages": [
        {
            "role": "user",
            "content": prompt
        }
    ],
    "stream": False
}

headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# timeout -> context가 길어 5초로는 부족, 60초로 늘려줌
with httpx.Client(timeout=60.0) as client:
    response = client.post(base_url, headers=headers, json=payload)

# 응답 결과 출력
print(response.status_code)
print(response.json())

200
{'id': '52a3bc67-ed82-4a63-ade0-055299888f94', 'object': 'chat.completion', 'created': 1752641648, 'model': 'solar-pro-250422', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '무인 교통단속용 장비의 설치 및 관리에 대해 문의하셨습니다. 도로교통법 제4조의2 ①항에 따라 지방경찰청장, 경찰서장 또는 시장 등은 교통법규 위반 사실을 기록하고 증명하기 위해 무인 교통단속용 장비를 설치 및 관리할 수 있습니다. 또한, 무인 교통단속용 장비의 철거 또는 원상회복 등에 대해서는 제3조 제4항부터 제6항까지의 규정을 준용하며, 이 경우 교통안전시설은 무인 교통단속용 장비로 간주됩니다.'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 1834, 'completion_tokens': 113, 'total_tokens': 1947}, 'system_fingerprint': None}


In [62]:
data = response.json()

print(data['choices'][0]['message']['content'])

무인 교통단속용 장비의 설치 및 관리에 대해 문의하셨습니다. 도로교통법 제4조의2 ①항에 따라 지방경찰청장, 경찰서장 또는 시장 등은 교통법규 위반 사실을 기록하고 증명하기 위해 무인 교통단속용 장비를 설치 및 관리할 수 있습니다. 또한, 무인 교통단속용 장비의 철거 또는 원상회복 등에 대해서는 제3조 제4항부터 제6항까지의 규정을 준용하며, 이 경우 교통안전시설은 무인 교통단속용 장비로 간주됩니다.
